## Importing the libraries

In [1]:
from google.colab import drive
drive.mount ("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import  pandas as pd
import  seaborn as sns
import  scipy.stats as stats
import  matplotlib.pyplot as plt

# Grouping forest-compatible products by HS4

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Data Competitions/BCG Gamma 2022/Dados - GAMMA Challenge 2022/Dados Extras/produtos_compativeis_com_a_floresta.csv', delimiter=';', dtype = {'hs6': "str"})
data.set_index('id', inplace=True)
data

,hs6,description,brazilian_exports,world_exports,brazilian_market_share
id,,,,,
1,090411,"Pimenta (do gênero piper), seca, não triturad...","107.929.737,00","1.488.534.670,00","7,25%"
2,030389,"Outros peixes, exceto fígados, ovas e sêmen","33.123.476,00","4.053.050.315,00","0,82%"
3,151110,"Óleos de dendê, em bruto","27.492.944,00","8.699.395.419,00","0,32%"
4,200989,Suco (sumo) de qualquer outra fruta ou produto...,"24.424.377,00","2.551.701.372,00","0,96%"
5,030572,"Cabeças, caudas e bexigas natatórias, de peixes","19.654.863,00","427.154.174,00","4,60%"
...,...,...,...,...,...
60,070490,"Couves, repolhos e produtos comestíveis semelh...","15.681,00","1.862.922.540,00","0,00%"
61,080132,"Castanha de caju, fresca ou seca, sem casca","13.796,00","4.925.260.480,00","0,00%"
62,030499,"Carnes de outros peixes, exceto filés, mesmo p...","12.253,00","2.098.727.354,00","0,00%"


In [4]:
# Products to be removed: non edible and related to fishing
non_comestible_ids = [9, 20, 21, 22, 23, 25, 41, 45, 50, 54]
fish_product_ids = [2, 5, 9, 11, 18, 31, 35, 51, 58, 62, 63, 13, 17, 37, 38, 46]

removable_ids = non_comestible_ids + fish_product_ids
data.drop(index=removable_ids, inplace=True)

In [5]:
data['hs4'] = data.apply(lambda row : row['hs6'][:4], axis=1)

In [6]:
# Group by HS4, concatenating descriptions
hs4_data = data.groupby(['hs4'],as_index=False).agg(lambda x : ' / '.join(x))[['hs4', 'description']]

# Value by product

In [7]:
# Source: WITS World Bank
# Query: Brazilian exports in 2022
value_by_product = pd.read_csv('/content/drive/MyDrive/Data Competitions/BCG Gamma 2022/Dados - GAMMA Challenge 2022/Dados Extras/valor_por_produto.csv', dtype = {'ProductCode': "str"})

In [8]:
# Select by HS4
value_by_product['hs4'] = value_by_product.apply(lambda row : row['ProductCode'][:4], axis=1)

In [9]:
# Group by HS4 summing values
value_by_product_hs4 = value_by_product.groupby(['hs4'],as_index=False).sum()

In [10]:
# Produces value by kg
value_by_product_hs4['per_kg'] = value_by_product_hs4['TradeValue in 1000 USD'] * 1000 / valor_por_produto_hs4['NetWeight in KGM']

In [11]:
value_by_product_hs4[['hs4', 'per_kg']]

,hs4,per_kg
0,0302,6.117393
1,0303,4.301257
2,0304,8.591435
3,0305,31.447248
4,0306,35.299053
5,0307,7.655053
6,0409,3.461353
7,0704,0.442514
8,0712,1.484810
9,0801,5.555858


In [12]:
product_value_map = dict(zip(value_by_product_hs4['hs4'], value_by_product_hs4['per_kg']))

In [13]:
hs4_data['hs4'].map(product_value_map)

0      3.461353
1      0.442514
2      1.484810
3      5.555858
4      0.934449
5      2.257296
6     22.344586
7      2.542738
8      3.326999
9      5.688014
10     9.269575
11    12.727154
12     0.353227
13     8.411294
14     5.290811
15     1.396096
16     1.904338
17     3.964849
18     3.916125
19     0.952838
20     2.173102
21     1.593005
22     6.280618
Name: hs4, dtype: float64

In [14]:
hs4_data['value_per_kg'] = hs4_data['hs4'].map(product_value_map)

In [15]:
hs4_data

,hs4,description,value_per_kg
0,0409,Mel natural,3.461353
1,0704,"Couves, repolhos e produtos comestíveis semelh...",0.442514
2,0712,"Cebolas secas, inclusive em pedaços ou fatias,...",1.484810
3,0801,"Castanha-do-pará, fresca ou seca, sem casca / ...",5.555858
4,0804,"Goiabas, mangas e mangostões, frescos ou secos...",0.934449
5,0811,"Outras frutas congeladas, não cozidas ou cozid...",2.257296
6,0813,Pêras e outras frutas secas,22.344586
7,0901,"Café não torrado, não descafeinado",2.542738
8,0904,"Pimenta (do gênero piper), seca, não triturad...",3.326999
9,0907,"Cravo-da-índia (frutos, flores e pedúnculo) nã...",5.688014


In [16]:
hs4_data.to_csv('/content/drive/MyDrive/Data Competitions/BCG Gamma 2022/Dados - GAMMA Challenge 2022/Dados Extras/produtos_hs4_filtrados_com_preco.csv', index=False)